# Prepare notebook

In [1]:
%load_ext dotenv
%load_ext autoreload
%autoreload 2
import os
# move current working directory up two levels to root
# not pretty but this is a notebook
# don't run this cell more than once or you'll move another two directories up, which wouldn't be good
os.chdir(os.pardir); os.chdir(os.pardir)
print('Current working directory is %s' % os.getcwd())
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from src import utilities
import settings
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from collections import OrderedDict
import copy
import re
import feedparser
import pubmed_parser
import time
import slate
import re
import pickle

Current working directory is E:\Users\Peter_Rasmussen\gh\multivac
Directory data\raw already exists
Directory data\interim already exists
Directory data\processed already exists
Directory data\raw\arxiv already exists
Directory data\raw\pubmed already exists
Directory data\raw\springer already exists


# Arxiv

In [65]:
import warnings
warnings.filterwarnings('ignore')
import pickle

In [74]:
# load metadata
with open(settings.metadata_dir / 'arxiv.pkl', 'rb') as f:
    arxiv_metadata_ = pickle.load(f)

'1511.08572v3'

In [ ]:
# we'll just add the text to a new arxiv object, an ordered dict keyed on doi or other id
arxiv_data = OrderedDict()

arxiv_srcs = [settings.arxiv_dir / x for x in os.listdir(settings.arxiv_dir)]

for ix, article_metadata in enumerate(arxiv_metadata_[109:]):
    fn = article_metadata['fn']
    # define key, usually the doi
    k = article_metadata['id'].split('/abs/')[-1]
    
    # initialize temp dictionary
    temp = OrderedDict()
    temp['metadata'] = copy.deepcopy(article_metadata)
    src = settings.arxiv_dir / fn
    try:
        # try to open file
        with open(src, 'rb') as f:
            doc = slate.PDF(f)

        # get text: strip out newlines and extra spaces
        doc = ' '.join([' '.join(x.split()) for x in doc])
        text = doc.split(' Abstract ')[-1].split(' Acknowledgments ')[0].split(' ∗ ∗ ∗ ')[0].strip()

        # add text to temp dict
        temp['data'] = text
        
    except:  #  PDFSyntaxError
        temp['data'] = None
    
    arxiv_data[k] = temp

In [84]:
len(arxiv_data)

109

In [81]:
arxiv_data[k]['data']

'–Studies on how to model the interplay between diseases and behavioral responses (so-called coupled disease-behavior interaction) have attracted increasing attention. Owing to the lack of obvious clinical evidence of diseases, or the incomplete information related to the disease, the risks of infection cannot be perceived and may lead to inappropriate behavioral responses. Therefore, how to quantitatively analyze the impacts of asymptomatic infection on the interplay between diseases and behavioral responses is of particular importance. In this Letter, under the complex network framework, we study the coupled disease-behavior interaction model by dividing infectious individuals into two states: U-state (without evident clinical symptoms, labelled as U) and I-state (with evident clinical symptoms, labelled as I). A susceptible individual can be infected by U- or I-nodes, however, since the U-nodes cannot be easily observed, susceptible individuals take behavioral responses only when th

In [61]:
len(doc.split(' Acknowledgments '))

2

In [57]:
'and \n the  dog'.split()

['and', 'the', 'dog']

In [53]:
doc[-24]

'Fourth, we have not considered dynamics of social networks (Volz and Meyers, 2007), except that we have conﬁrmed that the main results are qualitatively the same at two time points (see Appendix for the other data set). In reality, the members of a hospital change on a very short time scale due to, for example, admission and discharge of patients (Bootsma et al., 2006; Liljeros et al., 2007) In conclusion, the present study examined the spread of nosocomial infection in explicitly structured and observation-based social networks in a hospital, emphasizing the importance of controlling medical doctors as a potential ma jor vector of diseases. Despite the necessity of various improvements, we believe that our exercise highlights speciﬁc hierarchical structure of social networks of hospitals and associated disease spreading patterns. Acknowledgments We thank Hiroshi Nishiura and Takashi Ichinomiya for critical reading of the manuscript and Yuki Yasuda for helpful comments on this work. N

In [40]:
' '.join(doc)

'Abstract Nosocomial infection (i.e. infection in healthcare facilities) raises a serious public health problem, as implied by the existence of pathogens characteristic to healthcare facilities such as methicillin-resistant Staphylococcus aureus and hospital-mediated out- breaks of inﬂuenza and severe acute respiratory syndrome. For general communities, epidemic modeling based on social networks is being recognized as a useful tool. How- ever, disease propagation may occur in a healthcare facility in a manner diﬀerent from that in a urban community setting due to diﬀerent network architecture. We simulate stochastic susceptible-infected-recovered dynamics on social networks, which are based on observations in a hospital in Tokyo, to explore eﬀective containment strategies against nosocomial infection. The observed social networks in the hospital have hierarchical and modular structure in which dense substructure such as departments, wards, and rooms, are globally but only loosely conne

# Springer

In [17]:
li = ['sir model', 'susceptible infected recovered', 'irSIR model']
make_q = lambda li: '(' + ' OR '.join(['"' + s + '"' for s in li]) + ')'
q = make_q(li)

base = 'http://api.springernature.com/openaccess/json?q='
url = base + q
params = {'source': 'springer', 'openaccess': 'true', 'api_key': springer_api_key, 'p': 20, 's': 1}
params_ = copy.deepcopy(params)
r = requests.get(url, params_)
springer_metadata = []

while True:
    r = requests.get(url, params_)
    if len(r.json()['records']) == 0:
        break
    params_['s'] = params_['s'] + params_['p']
    springer_metadata += r.json()['records']
    time.sleep(wait_time)
print(len(springer_metadata))

382


In [ ]:
# 
for ix, md in enumerate(springer_metadata):
    fn = md['doi'].replace('/', '-')
    if len(fn) == 0:
        fn = md['identifier']
    r = requests.get(md['url'][0]['value'])
    html = bs(r.text).encode('utf-8').decode('utf-8')
    dst = settings.raw_dir / 'springer' / (fn + '.html')
    with open(dst, 'w', encoding='utf-8') as f:
        f.write(html)
    time.sleep(0.3)

In [32]:
soup = bs(r.text)

In [45]:
li = [x.contents[0] for x in soup.find('article').find_all('p', {'class':'Para'})]
li

['This paper investigates an SIS epidemic model with variable population size including a vaccination program. Dynamics of the endemic equilibrium of the model are obtained, and it will be shown that this equilibrium exists and is locally asymptotically stable when ',
 'The susceptible–infected–susceptible (SIS) model is one of the most well-known type of epidemic models. These models are appropriate for some infections, for instance, common cold and influenza, or bacterial diseases such as meningitis and cholera, or sexually transmitted diseases, that do not cause permanent immunity after recovery. To immunize individuals from infection and control the infectious diseases, vaccination is usually preferred because of its efficiency compared with other drug and non-drug interventions. There are many epidemic models [',
 'Here, the population has been divided into three subpopulations as susceptible, infected, and vaccinated individuals. The size of population in each class at time ',
 '

In [54]:
# print(soup.find('article').prettify())
li = [x.get_text() for x in soup.find('article').find_all('p')]
s = 'Springer Nature remains neutral '
li

['Mathematical Sciences',
 'December 2018, Volume 12, Issue\xa04,\n                        pp 313–320 | Cite as',
 'This paper investigates an SIS epidemic model with variable population size including a vaccination program. Dynamics of the endemic equilibrium of the model are obtained, and it will be shown that this equilibrium exists and is locally asymptotically stable when \\({\\mathcal {R}}_0 > 1\\). In this case, the disease uniformly persists, and moreover, using a geometric approach we conclude that the model is globally asymptotically stable under some conditions. Also, a numerical discussion is given to verify the theoretical results.',
 'The susceptible–infected–susceptible (SIS) model is one of the most well-known type of epidemic models. These models are appropriate for some infections, for instance, common cold and influenza, or bacterial diseases such as meningitis and cholera, or sexually transmitted diseases, that do not cause permanent immunity after recovery. To immu

In [30]:
text = pubmed_parser.parse_pubmed_paragraph(str((settings.raw_dir / 'springer' / 'test.xml').absolute()), all_paragraph=True)

Error: it was not able to read a path, a file-like object, or a string as an XML


XMLSyntaxError: Start tag expected, '<' not found, line 1, column 1 (<string>, line 1)

In [27]:
r = requests.get(springer_metadata[1]['url'][0]['value'])


{'feed': {'html': {'lang': 'en-gb', 'class': 'no-js'},
  'meta': {'name': 'msapplication-TileImage',
   'content': '/springerlink-static/1645515599/images/favicon/ic_launcher_xxhdpi.png'},
  'links': [{'rel': 'canonical',
    'href': 'https://link.springer.com/article/10.1007/s40096-018-0271-3',
    'type': 'text/html'},
   {'rel': 'shortcut icon',
    'href': '/springerlink-static/1645515599/images/favicon/favicon.ico',
    'type': 'text/html'},
   {'rel': 'icon',
    'sizes': '16x16 32x32 48x48',
    'href': '/springerlink-static/1645515599/images/favicon/favicon.ico',
    'type': 'text/html'},
   {'rel': 'icon',
    'sizes': '16x16',
    'type': 'image/png',
    'href': '/springerlink-static/1645515599/images/favicon/favicon-16x16.png'},
   {'rel': 'icon',
    'sizes': '32x32',
    'type': 'image/png',
    'href': '/springerlink-static/1645515599/images/favicon/favicon-32x32.png'},
   {'rel': 'icon',
    'sizes': '48x48',
    'type': 'image/png',
    'href': '/springerlink-static/16

# IEEE Xplore

In [ ]:
base = 'http://ieeexploreapi.ieee.org/api/v1/search/articles?'
url = base + q
params = {'max_records': 20, 'start_record': 1, 'querytext': q, 'apikey': ieee_api_key}
params_ = copy.deepcopy(params)

ieee_metadata = []
while True:
    r = requests.get(url, params_)
    if params_['start_record'] > r.json()['total_records']:
        break
    for article in r.json()['articles']:
        if i['access_type'] != 'LOCKED':
            ieee_metadata.append(article)
    params_['start_record'] = params_['start_record'] + params_['max_records']
    time.sleep(wait_time)
print(len(ieee_metadata))

# Pubmed (Entrez)
* https://marcobonzanini.com/2015/01/12/searching-pubmed-with-python/

In [25]:
# Download all of it
# get the ids
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc&term=%22sir+model%22&set=pmc-open&metadataPrefix=pmc&usehistory=y&retmax=2000'


Directory data already exists
Directory data\raw already exists
Directory data\raw\arxiv already exists
Directory data\raw\springer already exists
Directory data\raw\pubmed already exists
Directory data\interim already exists
Directory data\processed already exists
Directory data\processed\metadata already exists


In [ ]:
# search pubmed central for free full text articles containing selected query
terms = ['sir model', 'susceptible-infected-recovered']
replace = lambda s: s.replace(' ', '+')
quote = lambda s: '%22' + s + '%22'
terms = [quote(replace(s)) for s in terms]
term = 'term='+ '%28'+ '+OR+'.join(terms) + '%29'
fulltext = 'free+fulltext%5bfilter%5d'
retmax = 'retmax=2000'
base = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pmc'
params = {'retmax': 2000, 'email': }
url = base + '&' + term + '+' + fulltext + '&' + retmax
r = requests.get(url)
ids = [x.contents[0] for x in bs(r.text).find_all('id')]
ids[5]

In [ ]:
len(ids)

In [ ]:
pmc_articles = OrderedDict()
for i in ids:
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=' + str(i)
    r = requests.get(url, params={'id': i})
    xml = r.text
    pmc_id = 'pmc' + str(i)
    fn = (pmc_id + '.xml')
    path = settings.raw_dir / 'pubmed' / fn
    with open(path, 'w') as f:
        f.write(xml)
    time.sleep(0.5)

In [ ]:
if len(text) > 0:
    pmc_articles[doi] = {'metadata': metadata, 'text': text}

text = pubmed_parser.parse_pubmed_paragraph(str(path.absolute()), all_paragraph=True)
metadata = pubmed_parser.parse_pubmed_xml(str(path.absolute()))
doi = metadata.pop('doi')


In [ ]:
pmc_articles[doi]

In [ ]:
len(pmc_articles)

In [ ]:
pmc_articles

In [ ]:
paragraphs

In [ ]:
# r = requests.get(url, params={'id': i})


In [ ]:
src = r'E:\Users\Peter_Rasmussen\gh\multivac\data\raw\pubmed\pmc4760143'

In [ ]:
# the parsing answer: https://github.com/titipata/pubmed_parser


In [ ]:
url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pmc&id=6214536'
r = requests.get(url)

In [ ]:
pubmed_parser.parse_xml_web(ids[28], save_xml=False)

In [ ]:
xml = r.text

In [ ]:
imp

In [ ]:
import pubmed_parser

In [ ]:
dicts_out = pp.parse_pubmed_paragraph('data/6605965a.nxml', all_paragraph=False)

In [ ]:
print(xml.keys())

In [ ]:
oa_file_list_ = pd.read_csv('ftp://ftp.ncbi.nlm.nih.gov/pub/pmc/oa_file_list.csv')

In [ ]:
oa_file_list_['Article Citation'].unique()

In [ ]:
# tbd

url = 'https://www.ncbi.nlm.nih.gov/pmc/oai/oai.cgi?verb=ListRecords&metadataPrefix=pmc&term=%22sir+model%22+OR%22susceptible+-+infected+-+recovered%22&set=pmc-open'
r = requests.get(url)

In [ ]:
r.text[:1000]

# Old

In [ ]:



# def process_article_metadata(metadata):
#     """Extract metadata for one article and organize metadata into a dictionary.
#     Inputs:
#         metadata    String; Read in from API call
#     Outputs:
#         d  Dictionary of article metadata
#     """

#     # for each article extract and organize metadata
#     metadata = bs(metadata.strip(), 'lxml')

#     # each article's metadata contained in a dictionary
#     d = {}

#     # add single-value attributes
#     single_value_attributes = ['id', 'updated', 'published', 'title', 'summary', 'doi']
#     for single_value_attribute in single_value_attributes:
#         try:
#             v = metadata.find(single_value_attribute).contents[0].strip()
#         except AttributeError:
#             # some articles don't have doi numbers so fall back on arxiv doi
#             if single_value_attribute=='doi':
#                 v = d['id']
#             else:
#                 v = None
#         d[single_value_attribute] = v

#     # add multiple-value attributes and edge-case single-value attributes
#     d['arxiv:primary_category'] = metadata.find('arxiv:primary_category').attrs['term']
#     d['arxiv_categories'] = [x['term'] for x in metadata.find_all('category')]
#     d_links = {}
#     for link in metadata.find_all('link'):
#         if 'title' in link.attrs:
#             k, v = link.attrs['title'], link.attrs['href']
#             d_links[k] = v
#     d['links'] = d_links
#     d['authors'] = [x.contents[0] for x in metadata.find_all('name')]

#     return d


# def get_metadata_from_page(xml_text):
#     """
#     Usage of output:
#         As an OrderedDict, d_page_metadata has the usual dictionary functionality
#         It can also be accessed like a list using the approach below (Python 3 approach below):
#             items = list(d_page_metadata.items())
#             items[0]
#     """
    
#     # prep metadata returned by api query
#     articles_metadata = re.sub(' +', ' ', xml_text.replace('arxiv:doi', 'doi').replace('\n', ' ')).strip().split('<entry>')[1:]

#     # iterate over each article and extract and organize metadata
#     d_page_metadata = OrderedDict()
#     for article_metadata in articles_metadata:
#         v = copy.deepcopy(process_article_metadata(article_metadata))
#         k = v.pop('doi')
#         d_page_metadata[k] = v
    
#     return d_page_metadata



In [ ]:
# ps2ascii, gzip
import gzip
import os

In [ ]:
f = gzip.open(raw_src, 'rb')

In [ ]:
ps = f.read()

In [ ]:
f.close()

In [ ]:
raw_dir = r'E:\Users\Peter_Rasmussen\gh\multivac\data\raw\arxiv'
raw_src = os.path.join(raw_dir, '1411.2370v2.ps.gz')
# with gzip.open(raw_src, 'rb') as f:
#     ps = f.read()
    
def opener(filename):
    f = open(filename,'rb')
    if (f.read(2) == '\x1f\x8b'):
        f.seek(0)
        return gzip.GzipFile(fileobj=f)
    else:
        f.seek(0)
        return f
f = opener(raw_src)

In [ ]:
ps = f.read()
f.close()

In [ ]:
with opener(raw_src) as f:
    ps = f.read().decode('iso-8859-1')

In [ ]:
len(ps)

In [ ]:
list(d_metadata.items())[0]

In [ ]:
params_

In [ ]:
n_results

In [ ]:
len(d_metadata.keys())

In [ ]:
next(a)

In [ ]:
print()
print(80 * '*')
print(xml_text)


In [ ]:

for link in article_metadata.find_all('link'):
    pass

In [ ]:

if 'author' in link.attrs:
    k, v = link.attrs['title'], link.attrs['href']
    d_links[k] = v

In [ ]:
link.attrs

In [ ]:
v

In [ ]:
link

In [ ]:
link.attrs

In [ ]:
article_metadata.find_all('link')[0].attrs

In [ ]:

summary = ''.join(soup.find('summary').contents).strip().replace('\n', ' ')
title = ''.join(soup.find('title').contents).strip().replace('\n', ' ')
title

In [ ]:

soup.find_all('title')

In [ ]:
'http://export.arxiv.org/api/query?search_query=all:electron&start=0&max_results=10'

In [ ]:
import xml

In [ ]:
e = xml.etree.ElementTree.parse('xml.xml')

In [ ]:
e.findall('title')

In [ ]:
bf.data(r.text)

In [ ]:
r.text

In [ ]:
import urllib

base = 'http://export.arxiv.org/api/'
method = 'query'
search_term = 'electron'
parameters
parameters = 'search_query=all:' + electron&start=0&max_results=10'
query = base + method + '?' + parameters
data = urllib.request.urlopen(query).read()
print(data)

In [ ]:
data.